In [1]:
import pdfplumber
import pandas as pd
import os

sections = []
current_title = None
current_content = ""
root_title = None  # ilk başlığı tutacak

def is_close_color(c, target=(0.1882, 0.3176, 0.3922), tol=0.01):
    if not c or len(c) != 3:
        return False
    return all(abs(ci - ti) < tol for ci, ti in zip(c, target))

# PDF'i işle
with pdfplumber.open("ornek.pdf") as pdf:
    for page in pdf.pages:
        lines = {}
        for char in page.chars:
            top = round(char["top"], 1)
            if top not in lines:
                lines[top] = []
            lines[top].append(char)
        
        for top in sorted(lines.keys()):
            line_chars = lines[top]
            text = "".join(c["text"] for c in line_chars).strip()
            sizes = set(c["size"] for c in line_chars)  # size yine çıkarılıyor
            colors = set(c["non_stroking_color"] for c in line_chars)

            # sadece color kontrolü
            correct_color = any(is_close_color(c) for c in colors)

            if correct_color:
                # ilk başlık geldiğinde root_title olarak sakla
                if root_title is None:
                    root_title = text
                else:
                    text = f"{root_title}>{text}"

                # önceki başlık varsa kaydet
                if current_title:
                    sections.append({
                        "title": current_title,
                        "content": current_content.strip()
                    })
                current_title = text
                current_content = ""
            else:
                current_content += text + " "

# Son başlığı ekle
if current_title:
    sections.append({"title": current_title, "content": current_content.strip()})

# Yeni dataframe
new_df = pd.DataFrame(sections)

# Eski CSV varsa oku, yoksa boş DataFrame
if os.path.exists("data.csv"):
    old_df = pd.read_csv("data.csv")
    combined_df = pd.concat([old_df, new_df], ignore_index=True)
else:
    combined_df = new_df

# Kaydet
combined_df.to_csv("data.csv", index=False, encoding="utf-8")
print(combined_df)

                                                title  \
0                                     Getting Started   
1   Getting Started>Setting Up Your 3DEXPERIENCE P...   
2   Getting Started>Starting the 3DEXPERIENCE Plat...   
3   Getting Started>Discovering Roles, Apps, Busin...   
4               Getting Started>Accessing a Dashboard   
5     Getting Started>Opening an App from the Compass   
6   Getting Started>Opening a Native App from a De...   
7   Getting Started>Selecting a User Profile for D...   
8   Getting Started>Selecting a Mouse Profile for ...   
9                Getting Started>Selecting a Language   
10                       Getting Started>Getting Help   
11                       Getting Started>Getting Help   
12  Getting Started>Displaying Tooltips and Long H...   
13            Getting Started>Getting Contextual Help   
14    Getting Started>Using the User Assistance Panel   
15                Getting Started>Accessing Tutorials   
16  Getting Started>Accessing I

In [2]:
import pdfplumber
import pandas as pd
import os

# ✅ Ayarlar
PDF_ROOT_DIR = "pdf_folder"
OUTPUT_CSV = "data.csv"
TARGET_COLOR = (0.1882, 0.3176, 0.3922)
MIN_TITLE_GAP = 15  # başlıklar arası dikey mesafe farkı (yani çok yakınsa iptal edilecek)

# 📄 CSV birleştirme için
all_sections = []

# 📂 Klasördeki tüm PDF dosyalarını tara
for root, _, files in os.walk(PDF_ROOT_DIR):
    for file in sorted(files):
        if file.lower().endswith(".pdf"):
            pdf_path = os.path.join(root, file)
            print(f"🔍 İşleniyor: {pdf_path}")

            sections = []
            root_title = None
            current_title = None
            current_content = ""
            current_size = None

            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    lines = {}
                    for char in page.chars:
                        top = round(char["top"], 1)
                        lines.setdefault(top, []).append(char)

                    sorted_tops = sorted(lines.keys())

                    # 🔍 Başlık adayı satırları topla
                    candidates = []
                    for idx, top in enumerate(sorted_tops):
                        line_chars = lines[top]
                        text = "".join(c["text"] for c in line_chars).strip()
                        sizes = set(c["size"] for c in line_chars)
                        colors = set(c["non_stroking_color"] for c in line_chars)

                        has_correct_color = TARGET_COLOR in colors
                        has_correct_size = any(s > 10 for s in sizes)

                        is_title = has_correct_color and has_correct_size
                        candidates.append({
                            "top": top,
                            "text": text,
                            "line_chars": line_chars,
                            "is_title": is_title
                        })

                    # 🔁 Gerçek başlıkları belirle
                    valid_titles = []
                    for i, item in enumerate(candidates):
                        if not item["is_title"]:
                            continue

                        prev = candidates[i - 1] if i > 0 else None
                        next = candidates[i + 1] if i + 1 < len(candidates) else None

                        too_close_prev = prev and prev["is_title"] and abs(item["top"] - prev["top"]) < MIN_TITLE_GAP
                        too_close_next = next and next["is_title"] and abs(next["top"] - item["top"]) < MIN_TITLE_GAP

                        if too_close_prev or too_close_next:
                            item["is_title"] = False  # başlık olarak işleme
                        else:
                            valid_titles.append(item["top"])

                    # 🔁 Satırları işle
                    for top in sorted_tops:
                        line_chars = lines[top]
                        text = "".join(c["text"] for c in line_chars).strip()
                        sizes = set(c["size"] for c in line_chars)
                        colors = set(c["non_stroking_color"] for c in line_chars)

                        has_correct_color = TARGET_COLOR in colors
                        has_correct_size = any(s > 10 for s in sizes)

                        is_title = has_correct_color and has_correct_size and top in valid_titles

                        if is_title:
                            if root_title is None:
                                root_title = text
                            else:
                                text = f"{root_title}>{text}"

                            if current_title:
                                sections.append({
                                    "title": current_title,
                                    "content": current_content.strip(),
                                    "correct_color": True,
                                    "correct_size": True,
                                    "source_pdf": file,
                                    "font_size": current_size if current_size else ""
                                })

                            current_title = text
                            current_content = ""
                            size_vals = [c["size"] for c in line_chars]
                            current_size = sum(size_vals) / len(size_vals) if size_vals else ""
                        else:
                            current_content += text + " "

            if current_title:
                sections.append({
                    "title": current_title,
                    "content": current_content.strip(),
                    "correct_color": True,
                    "correct_size": True,
                    "source_pdf": file,
                    "font_size": current_size if current_size else ""
                })

            print(f"📘 {file} içinden çıkarılan topic sayısı: {len(sections)}")
            all_sections.extend(sections)

# 📊 DataFrame oluştur ve kaydet
new_df = pd.DataFrame(all_sections)

if os.path.exists(OUTPUT_CSV):
    old_df = pd.read_csv(OUTPUT_CSV)
    combined_df = pd.concat([old_df, new_df], ignore_index=True)
else:
    combined_df = new_df

combined_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✅ Tüm veriler kaydedildi: {OUTPUT_CSV}")

🔍 İşleniyor: pdf_folder\ornek10.pdf
📘 ornek10.pdf içinden çıkarılan topic sayısı: 17
🔍 İşleniyor: pdf_folder\ornek100.pdf


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


📘 ornek100.pdf içinden çıkarılan topic sayısı: 551
🔍 İşleniyor: pdf_folder\ornek101.pdf
📘 ornek101.pdf içinden çıkarılan topic sayısı: 32
🔍 İşleniyor: pdf_folder\ornek102.pdf
📘 ornek102.pdf içinden çıkarılan topic sayısı: 289
🔍 İşleniyor: pdf_folder\ornek103.pdf
📘 ornek103.pdf içinden çıkarılan topic sayısı: 7
🔍 İşleniyor: pdf_folder\ornek104.pdf
📘 ornek104.pdf içinden çıkarılan topic sayısı: 29
🔍 İşleniyor: pdf_folder\ornek105.pdf
📘 ornek105.pdf içinden çıkarılan topic sayısı: 32
🔍 İşleniyor: pdf_folder\ornek106.pdf
📘 ornek106.pdf içinden çıkarılan topic sayısı: 30
🔍 İşleniyor: pdf_folder\ornek107.pdf
📘 ornek107.pdf içinden çıkarılan topic sayısı: 16
🔍 İşleniyor: pdf_folder\ornek108.pdf
📘 ornek108.pdf içinden çıkarılan topic sayısı: 13
🔍 İşleniyor: pdf_folder\ornek109.pdf
📘 ornek109.pdf içinden çıkarılan topic sayısı: 20
🔍 İşleniyor: pdf_folder\ornek11.pdf
📘 ornek11.pdf içinden çıkarılan topic sayısı: 98
🔍 İşleniyor: pdf_folder\ornek110.pdf
📘 ornek110.pdf içinden çıkarılan topic sayıs

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


📘 ornek13.pdf içinden çıkarılan topic sayısı: 447
🔍 İşleniyor: pdf_folder\ornek130.pdf
📘 ornek130.pdf içinden çıkarılan topic sayısı: 39
🔍 İşleniyor: pdf_folder\ornek131.pdf
📘 ornek131.pdf içinden çıkarılan topic sayısı: 31
🔍 İşleniyor: pdf_folder\ornek132.pdf
📘 ornek132.pdf içinden çıkarılan topic sayısı: 16
🔍 İşleniyor: pdf_folder\ornek133.pdf
📘 ornek133.pdf içinden çıkarılan topic sayısı: 131
🔍 İşleniyor: pdf_folder\ornek134.pdf
📘 ornek134.pdf içinden çıkarılan topic sayısı: 6
🔍 İşleniyor: pdf_folder\ornek135.pdf
📘 ornek135.pdf içinden çıkarılan topic sayısı: 254
🔍 İşleniyor: pdf_folder\ornek136.pdf
📘 ornek136.pdf içinden çıkarılan topic sayısı: 710
🔍 İşleniyor: pdf_folder\ornek137.pdf
📘 ornek137.pdf içinden çıkarılan topic sayısı: 10
🔍 İşleniyor: pdf_folder\ornek138.pdf
📘 ornek138.pdf içinden çıkarılan topic sayısı: 294
🔍 İşleniyor: pdf_folder\ornek139.pdf
📘 ornek139.pdf içinden çıkarılan topic sayısı: 293
🔍 İşleniyor: pdf_folder\ornek14.pdf


...

📘 ornek367.pdf içinden çıkarılan topic sayısı: 1138
🔍 İşleniyor: pdf_folder\ornek368.pdf
📘 ornek368.pdf içinden çıkarılan topic sayısı: 15
🔍 İşleniyor: pdf_folder\ornek369.pdf
📘 ornek369.pdf içinden çıkarılan topic sayısı: 133
🔍 İşleniyor: pdf_folder\ornek37.pdf
📘 ornek37.pdf içinden çıkarılan topic sayısı: 342
🔍 İşleniyor: pdf_folder\ornek370.pdf


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


📘 ornek370.pdf içinden çıkarılan topic sayısı: 331
🔍 İşleniyor: pdf_folder\ornek371.pdf
📘 ornek371.pdf içinden çıkarılan topic sayısı: 25
🔍 İşleniyor: pdf_folder\ornek372.pdf
📘 ornek372.pdf içinden çıkarılan topic sayısı: 217
🔍 İşleniyor: pdf_folder\ornek373.pdf
📘 ornek373.pdf içinden çıkarılan topic sayısı: 3
🔍 İşleniyor: pdf_folder\ornek374.pdf
📘 ornek374.pdf içinden çıkarılan topic sayısı: 377
🔍 İşleniyor: pdf_folder\ornek375.pdf
📘 ornek375.pdf içinden çıkarılan topic sayısı: 83
🔍 İşleniyor: pdf_folder\ornek376.pdf
📘 ornek376.pdf içinden çıkarılan topic sayısı: 133
🔍 İşleniyor: pdf_folder\ornek377.pdf
📘 ornek377.pdf içinden çıkarılan topic sayısı: 161
🔍 İşleniyor: pdf_folder\ornek378.pdf
📘 ornek378.pdf içinden çıkarılan topic sayısı: 106
🔍 İşleniyor: pdf_folder\ornek379.pdf
📘 ornek379.pdf içinden çıkarılan topic sayısı: 70
🔍 İşleniyor: pdf_folder\ornek38.pdf
📘 ornek38.pdf içinden çıkarılan topic sayısı: 1
🔍 İşleniyor: pdf_folder\ornek380.pdf
📘 ornek380.pdf içinden çıkarılan topic sa

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


📘 ornek54.pdf içinden çıkarılan topic sayısı: 290
🔍 İşleniyor: pdf_folder\ornek55.pdf
📘 ornek55.pdf içinden çıkarılan topic sayısı: 79
🔍 İşleniyor: pdf_folder\ornek56.pdf
📘 ornek56.pdf içinden çıkarılan topic sayısı: 309
🔍 İşleniyor: pdf_folder\ornek57.pdf
📘 ornek57.pdf içinden çıkarılan topic sayısı: 110
🔍 İşleniyor: pdf_folder\ornek58.pdf
📘 ornek58.pdf içinden çıkarılan topic sayısı: 18
🔍 İşleniyor: pdf_folder\ornek59.pdf
📘 ornek59.pdf içinden çıkarılan topic sayısı: 33
🔍 İşleniyor: pdf_folder\ornek6.pdf
📘 ornek6.pdf içinden çıkarılan topic sayısı: 371
🔍 İşleniyor: pdf_folder\ornek60.pdf
📘 ornek60.pdf içinden çıkarılan topic sayısı: 105
🔍 İşleniyor: pdf_folder\ornek61.pdf
📘 ornek61.pdf içinden çıkarılan topic sayısı: 129
🔍 İşleniyor: pdf_folder\ornek62.pdf
📘 ornek62.pdf içinden çıkarılan topic sayısı: 217
🔍 İşleniyor: pdf_folder\ornek63.pdf
📘 ornek63.pdf içinden çıkarılan topic sayısı: 253
🔍 İşleniyor: pdf_folder\ornek64.pdf
📘 ornek64.pdf içinden çıkarılan topic sayısı: 205
🔍 İşleniy

In [ ]:
import pandas as pd

# CSV dosyasını oku
df = pd.read_csv("data.csv")

# Aynı title ve content'e sahip tekrar eden satırları grupla
duplicate_rows = df.duplicated(subset=["title", "content"], keep="false")

# Bu satırların sayısını yazdır
count = duplicate_rows.sum()
print(f"🔁 Aynı title ve content'e sahip satır sayısı: {count}")

🔁 Aynı title ve content'e sahip satır sayısı: 0


In [9]:
# İstenmeyen sütunları çıkar
columns_to_drop = ["correct_color", "correct_size", "source_pdf", "font_size"]
df = df.drop(columns=columns_to_drop, errors='ignore')  # errors='ignore' eksik sütun varsa hata vermez

In [ ]:
import pandas as pd

# veri.csv'yi oku
df_veri = pd.read_csv("veri.csv")

# mevcut df ile birleştir
df_birlesik = pd.concat([df, df_veri], ignore_index=True)

# Sonucu yeni dosyaya kaydet
df_birlesik.to_csv("birlesik_veri.csv", index=False)
print("✅ df ile veri.csv başarıyla birleştirildi ve birlesik_veri.csv olarak kaydedildi.")


✅ df ile veri.csv başarıyla birleştirildi ve birlesik_veri.csv olarak kaydedildi.


In [17]:
import pandas as pd
df=pd.read_csv("birlesik_veri.csv")
df.head()

,title,content
0,Getting Started,3DEXPERIENCE provides an easy-to-use interface...
1,Getting Started>Setting Up Your 3DEXPERIENCE P...,If you are the platform administrator you need...
2,Getting Started>Starting the 3DEXPERIENCE Plat...,Once you have been granted the required access...
3,"Getting Started>Discovering Roles, Apps, Busin...","You can discover all the roles, apps, business..."
4,Getting Started>Accessing a Dashboard,Dashboards let you gather and reuse content fr...


In [18]:
print(df.iloc[-10]["title"])

Installation and Setup > 3DEXPERIENCE Platform > 3DEXPERIENCE Configuration and Customization > 3DSpace Data Model > Unified Typing Reference Dictionary > Packages > XCADModeler Types


In [1]:
import pandas as pd

# Örneğin mevcut veri çerçevesi
new_df = pd.read_csv("data.csv")

# Yeni df oluştur: son 991 veriyi al
#new_df = combined_df.tail(991)


In [3]:
new_df.tail()

,title,content,correct_color,correct_size,source_pdf,font_size
57760,Systems Architecture>Object Naming and Types,The objects created in Preliminary Risk Assess...,True,True,ornek99.pdf,10.665
57761,Systems Architecture>Labels Configuration,Readers can refer to specific components in th...,True,True,ornek99.pdf,10.665
57762,Systems Architecture>Defining Time-Out Value,https://help.3ds.com/2024x/english/DSDoc/Front...,True,True,ornek99.pdf,10.665
57763,Systems Architecture>Attributes on Occurrences,Readers may need to define attributes on occur...,True,True,ornek99.pdf,10.665
57764,Systems Architecture>Custom Attributes for RAM...,"As an Administrator, you can use the Attribute...",True,True,ornek99.pdf,10.665


In [55]:
import pandas as pd

# Boşluk karakterlerini de boş sayabilmek için content'i temizle
new_df["content"] = new_df["content"].fillna("").astype(str).str.strip()

# Yeni DataFrame'e sadece şu satırları al:
# - Eğer title ilk kez görülüyorsa: her durumda dahil et
# - Eğer daha önce görüldüyse ama content boş değilse: dahil et
# - Eğer daha önce görüldüyse ve content boşsa: dahil etme

seen_titles = set()
clean_rows = []

for _, row in new_df.iterrows():
    title = row["title"]
    content = row["content"]

    if title not in seen_titles:
        clean_rows.append(row)
        seen_titles.add(title)
    else:
        if content.strip():  # boş değilse
            clean_rows.append(row)

# Yeni temiz DataFrame
clean_df = pd.DataFrame(clean_rows)

# Sonuç
print(f"🧹 Temizlenmiş veri satır sayısı: {clean_df.shape[0]}")

# CSV'ye kaydetmek istersen:
# clean_df.to_csv("veri_pdf_cleaned.csv", index=False)


🧹 Temizlenmiş veri satır sayısı: 682


C:\Users\sagla\AppData\Local\Temp\ipykernel_14488\2781298192.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["content"] = new_df["content"].fillna("").astype(str).str.strip()


In [62]:
# content'i boş olan ilk 100 satırı filtrele
empty_content_rows = new_df[new_df["content"].fillna("").str.strip() == ""]

# İlk 100 satırı al
empty_content_rows_head = empty_content_rows

# CSV olarak kaydet
empty_content_rows_head.to_csv("a.csv", index=False, encoding="utf-8")

print("✅ İlk 100 boş içerikli satır a.csv dosyasına kaydedildi.")


✅ İlk 100 boş içerikli satır a.csv dosyasına kaydedildi.


In [32]:
# Aynı title’dan birden fazla varsa tümünü sil (tamamen benzersizler kalsın)
df_unique = new_df[~new_df["title"].duplicated(keep=False)]

# Sonuç
print(f"✅ Eşsiz title'lara sahip kalan satır sayısı: {df_unique.shape[0]}")

# Yeni CSV'ye istersen kaydet
# df_unique.to_csv("veri_pdf_unique.csv", index=False)

✅ Eşsiz title'lara sahip kalan satır sayısı: 317


In [ ]:
# 🎯 Hedef font size
hedef_size = 10.665000408824994

# 📊 Bu font size'a sahip satırları filtrele
new_df = new_df[new_df["font_size"] == hedef_size].copy()

# ✅ Sonucu göster
print(f"🎯 Font size {hedef_size} olan satır sayısı: {hedef_df.shape[0]}")

In [24]:
filtered_df.tail

<bound method NDFrame.tail of                                                   title  \
6039                      Information Intelligence Apps   
6040      Information Intelligence Apps>B.I. Essentials   
6041           Information Intelligence Apps>What's New   
6042  Information Intelligence Apps>About B.I. Essen...   
6044  Information Intelligence Apps>Social and Colla...   
...                                                 ...   
7019            Information Intelligence Apps>Detecting   
7020              Information Intelligence Apps>Savings   
7021              Information Intelligence Apps>Tracked   
7022                 Information Intelligence Apps>Task   
7023          Information Intelligence Apps>Progression   

                                                content correct_color  \
6039  Powered by the 3DEXPERIENCE platform, Informat...          True   
6040  B.I. Essentials is an Information Intelligence...          True   
6041         There are no enhancements in 

In [14]:
import pandas as pd

# Boş veya sadece boşluk olan content satırlarını filtrele
new_df = new_df[new_df['content'].notna()]  # NaN olmayanlar
new_df = new_df[new_df['content'].str.strip() != ""]  # sadece boşluk içerenleri de çıkar

print("✅ Boş content'ler silindi ve dosya güncellendi.")


✅ Boş content'ler silindi ve dosya güncellendi.


In [16]:
# "Information Intelligence Apps>" ile başlayan title'ları filtrele
filtered = new_df[new_df['title'].str.startswith("Information Intelligence Apps>")]

# Sonucu yazdır
print(f"🔢 'Information Intelligence Apps>' ile başlayan satır sayısı: {len(filtered)}")

🔢 'Information Intelligence Apps>' ile başlayan satır sayısı: 550


In [8]:
combined_df.info

<bound method DataFrame.info of                                                   title  \
0                                       Getting Started   
1     Getting Started>Setting Up Your 3DEXPERIENCE P...   
2     Getting Started>Starting the 3DEXPERIENCE Plat...   
3     Getting Started>Discovering Roles, Apps, Busin...   
4                 Getting Started>Accessing a Dashboard   
...                                                 ...   
4629                       Fluid Systems>Specifications   
4630                       Fluid Systems>Administration   
4631  Fluid Systems>Data Setup: Fluidic Engineering ...   
4632  Fluid Systems>Accessing Engineering Specificat...   
4633  Fluid Systems>Types for Engineering Specificat...   

                                                content correct_color  \
0     3DEXPERIENCE provides an easy-to-use interface...           NaN   
1     If you are the platform administrator you need...           NaN   
2     Once you have been granted the req

In [ ]:
import pandas as pd
df=pd.read_csv("veri_pdf.csv")
print(df.iloc[-2]["content"])